In [1]:
import numpy as np
import sh
import sys
import os
import csv
import glob

def main():
    # mp3_dir = "mp3"

    feature_dir = "feature"
    model_dir = "model"
    prediction_wav_dir = "prediction_wav"
    read_dir = "read"
    padding_dir = "padding"
    redimension_dir = "redimension"
    window_dir = "window"
    sample_csv_dir = "sample_csv"
    wav_dir = "wav"
    dir_name = feature_dir +" "+ model_dir +" "+ prediction_wav_dir +" "+ read_dir +" "+ padding_dir +" "+ redimension_dir +" "+ sample_csv_dir  +" "+ wav_dir+" "+ window_dir
    sh.run("mkdir -p "+ dir_name)
    #sh.run("rm -rf "+str_name)

    # mp3_to_wav.run(mp3_file, wav_dir)
    wav_train__dir = wav_dir + "/" + "train"
    wav_prediction_dir = wav_dir + "/" + "prediction"
    csv_train_dir = sample_csv_dir + "/" + "train"
    csv_prediction_dir = sample_csv_dir + "/" + "prediction"
    redim_train_dir = redimension_dir+ "/" + "train"
    redim_test_dir = redimension_dir+ "/" +"test"
    sub_dir = wav_train__dir +" "+ wav_prediction_dir +" "+ csv_train_dir +" "+ csv_prediction_dir +" "+redim_train_dir +" "+redim_test_dir
    sh.run("mkdir -p "+ sub_dir)

    labels = []
    for i, f in enumerate(glob.glob(sample_csv_dir + os.sep +'*')):
        # print f
        for j,f_sub in enumerate(glob.glob(f + os.sep +'*.csv')):
            # print f_sub
            sample_file = f_sub
            with open(sample_file, 'rb') as csvfile:

            	reader = csv.reader(csvfile, delimiter=',', quotechar='|')

                for k, row in enumerate(reader):
                    labels.append(row[2])


    print np.unique(labels)
    for l in np.unique(labels):
        print l
        sh.run("mkdir -p "+ read_dir +"/"+ l)
        sh.run("mkdir -p "+ padding_dir +"/"+ l)
        # sh.run("mkdir -p "+ redimension_dir +"/"+ l)
        sh.run("mkdir -p "+ redim_train_dir +"/"+ l)
        sh.run("mkdir -p "+ redim_test_dir +"/"+ l)
        sh.run("mkdir -p "+ prediction_wav_dir +"/"+ l)


if __name__ == '__main__':
	main()

mkdir -p feature model prediction_wav read padding redimension sample_csv wav window
mkdir -p wav/train wav/prediction sample_csv/train sample_csv/prediction redimension/train redimension/test
['Ba' 'Bm' 'Eg']
Ba
mkdir -p read/Ba
mkdir -p padding/Ba
mkdir -p redimension/train/Ba
mkdir -p redimension/test/Ba
mkdir -p prediction_wav/Ba
Bm
mkdir -p read/Bm
mkdir -p padding/Bm
mkdir -p redimension/train/Bm
mkdir -p redimension/test/Bm
mkdir -p prediction_wav/Bm
Eg
mkdir -p read/Eg
mkdir -p padding/Eg
mkdir -p redimension/train/Eg
mkdir -p redimension/test/Eg
mkdir -p prediction_wav/Eg


In [3]:
from datetime import datetime, timedelta, tzinfo

class FixedOffset(tzinfo):
    """offset_str: Fixed offset in str: e.g. '-0400'"""
    def __init__(self, offset_str):
        sign, hours, minutes = offset_str[0], offset_str[1:3], offset_str[3:]
        offset = (int(hours) * 60 + int(minutes)) * (-1 if sign == "-" else 1)
        self.__offset = timedelta(minutes=offset)
        # NOTE: the last part is to remind about deprecated POSIX GMT+h timezones
        # that have the opposite sign in the name;
        # the corresponding numeric value is not used e.g., no minutes
        '<%+03d%02d>%+d' % (int(hours), int(minutes), int(hours)*-1)
    def utcoffset(self, dt=None):
        return self.__offset
    def tzname(self, dt=None):
        return self.__name
    def dst(self, dt=None):
        return timedelta(0)
    def __repr__(self):
        return 'FixedOffset(%d)' % (self.utcoffset().total_seconds() / 60)

def read(csv_train_dir):
    segProjet = []
    segSite = []
    segStart = []
    #segEnd = []
    segLabel = []
    duration = []
    segQuality = []
    for j,sample_file in enumerate(glob.glob(csv_train_dir + os.sep +'*.csv')):
        print sample_file
        with open(sample_file, 'rb') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for j, row in enumerate(reader):
                segProjet.append(row[0])
                segSite.append(row[1])
                date_with_tz = row[3]
                #print date_with_tz

                start = datetime.strptime(date_with_tz, "%Y-%m-%dT%H:%M:%S.%f")

                #start = datetime.datetime.strptime(row[3], "%Y-%m-%dT%H:%M:%S")
                segStart.append(start)
                #segEnd.append(row[4])
                end = datetime.strptime(row[4], "%Y-%m-%dT%H:%M:%S.%f")
                segLabel.append(row[2])
                dur = (end - start).total_seconds()
                duration.append(dur)

                segQuality.append(row[5])
                #print dur
        #print segProjet,segSite,segStart,duration,segLabel,segQuality

    return segProjet,segSite,segStart,duration,segLabel,segQuality

def date_type(wav_dir,segProjet,segSite,segStart,duration,segLabel,segQuality,read_dir):
    print max(duration)
    print min(duration)
    for i, f in enumerate(glob.glob(wav_dir + os.sep +'*.wav')):               # for each WAV file
        wavFile = f
        print os.path.splitext(wavFile)[0]
        waveFile_name = (os.path.splitext(wavFile)[0]).split(os.sep)[2]
        print waveFile_name
        date1 = waveFile_name.split("_")[3]
        date2 = (waveFile_name.split("_")[4]).split(".")[0]
        #print date1,date2
        temp1 = []
        temp2 = []
        for i in range(len(date1)):
            #print date1[i], '(%d)' %i
            temp1.append(date1[i])
        date1 = "20" + temp1[0] + temp1[1] +"-"+ temp1[2] + temp1[3] +"-"+ temp1[4]+temp1[5]
        print ("date1 is ",date1)

        for j in range(len(date2)):
            #print date2[j], '(%d)' %j
            temp2.append(date2[j])
        date2 = temp2[0] + temp2[1] +":"+ temp2[2] + temp2[3] +":"+ temp2[4]+temp2[5]
        #print date2

        date_str = date1 +" "+ date2
        #print data_str

        start_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")

        print ("start_date is ",start_date)
        
        cut(wavFile, segProjet,segSite,segStart,duration,segLabel,segQuality, read_dir,start_date)


        
def cut(wavFile,segProjet,segSite,segStart,duration,segLabel,segQuality, read_dir,start_date):
    # date2 = datetime.strptime(date2, "%H:%M:%S")
    #print("date2 is ",date2)
    cmd = """ffmpeg -i """ + wavFile + """ 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'"""
    rs = sh.run(cmd, True)

    duration_in_wavFile = rs.stdout()
    print("the duration of wavfile is ", duration_in_wavFile)
    # print type(duration_in_wavFile)
    # print type(duration)

    w = os.path.splitext(wavFile)[0]
    projet_wav = w.split("_")[0]
    site_wav = w.split("_")[1]

    good_projet = []
    good_site = []
    good_species = []
    good_start = []
    good_cut_point = []
    good_duration = []
    good_quality = []
    for i in range(0, len(segStart)):
        if i < len(segStart) - 1:
            cut_point = (segStart[i] - start_date).total_seconds()
            #print cut_point
            #print type(cut_point)

            if cut_point >= 0 and float(cut_point) <= float(duration_in_wavFile):
                good_projet.append(segProjet[i])
                good_site.append(segSite[i])
                good_species.append(segLabel[i])

                temp_start = segStart[i].strftime("%Y-%m-%dT%H:%M:%S.%f")
                temp_start = temp_start[:-5]
                good_start.append(temp_start)

                good_cut_point.append(cut_point)
                good_duration.append(duration[i])
                good_quality.append(segQuality[i])
    print good_cut_point,str(good_duration)
    print good_start
    encode_good_speices = np.unique(good_species)
    for j in range(0,len(good_cut_point)):
        str_name = str(good_projet[j]) +"_"+ str(good_site[j]) +"_"+ str(good_start[j]) +"."+ str(good_species[j]) +"."+ str(good_quality[j])
        for s in encode_good_speices:
            if str(good_species[j]) == s and str(good_projet[j]==projet_wav) and str(good_site[j])==site_wav:
                cmd = "ffmpeg -ss " + str(good_cut_point[j]) + " -t " + str(good_duration[j])+ " -i " + wavFile + " " + read_dir + "/" + s + "/" + str_name + ".wav"
                sh.run(cmd)        
        
def clean(file_dir):
    for i, f in enumerate(glob.glob(file_dir + os.sep +'*')):
        # print f
        cmd = "rm -rf " + f  + "/*.wav"
        sh.run(cmd)

if __name__ == '__main__':

    csv_train_dir = "sample_csv/train"

    wav_dir = 'wav/train'
    read_dir = "read"
    clean(read_dir)
    segProjet,segSite,segStart,duration,segLabel,segQuality = read(csv_train_dir)
    date_type(wav_dir,segProjet,segSite,segStart,duration,segLabel,segQuality,read_dir)

rm -rf read/Bm/*.wav
rm -rf read/Eg/*.wav
rm -rf read/Ba/*.wav
sample_csv/train/HAT_A_LF_dev.csv
sample_csv/train/WAT_NC_LF_dev.csv
sample_csv/train/WAT_BP_dev.csv
sample_csv/train/WAT_HZ_dev.csv
43.1
1.0
wav/train/WAT_HZ_01_160124_170000.df100.x
WAT_HZ_01_160124_170000.df100.x
('date1 is ', '2016-01-24')
('start_date is ', datetime.datetime(2016, 1, 24, 17, 0))
ffmpeg -i wav/train/WAT_HZ_01_160124_170000.df100.x.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '21600')
[] []
[]
wav/train/WAT_HZ_01_160217_000000.df100.x
WAT_HZ_01_160217_000000.df100.x
('date1 is ', '2016-02-17')
('start_date is ', datetime.datetime(2016, 2, 17, 0, 0))
ffmpeg -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '79875')
[139.1, 204.3, 273.6, 336.6, 397.9, 450.2

ffmpeg -ss 204.3 -t 6.2 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:03:24.3.Bm.good.wav
ffmpeg -ss 273.6 -t 7.0 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:04:33.6.Bm.good.wav
ffmpeg -ss 336.6 -t 5.2 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:05:36.6.Bm.good.wav
ffmpeg -ss 397.9 -t 4.4 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:06:37.9.Bm.good.wav
ffmpeg -ss 450.2 -t 2.6 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:07:30.2.Bm.good.wav
ffmpeg -ss 521.6 -t 3.0 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:08:41.6.Bm.good.wav
ffmpeg -ss 533.3 -t 1.6 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:08:53.3.Bm.good.wav
ffmpeg -ss 677.9 -t 5.6 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T00:11:17.9.Bm.good.wav
ffmpeg -ss 697.5 -t 9.8 

ffmpeg -ss 6609.7 -t 5.4 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:50:09.7.Bm.good.wav
ffmpeg -ss 6671.6 -t 3.5 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:51:11.6.Bm.good.wav
ffmpeg -ss 6740.0 -t 3.5 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:52:20.0.Bm.poor.wav
ffmpeg -ss 6802.2 -t 4.4 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:53:22.2.Bm.good.wav
ffmpeg -ss 6811.8 -t 6.1 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:53:31.8.Bm.good.wav
ffmpeg -ss 6876.5 -t 4.0 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:54:36.5.Bm.good.wav
ffmpeg -ss 6941.5 -t 4.5 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:55:41.5.Bm.good.wav
ffmpeg -ss 7004.6 -t 4.0 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T01:56:44.6.Bm.good.wav
ffmpeg -ss 7014.

ffmpeg -ss 12344.5 -t 3.6 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:25:44.5.Bm.good.wav
ffmpeg -ss 12410.1 -t 3.3 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:26:50.1.Bm.good.wav
ffmpeg -ss 12477.8 -t 3.1 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:27:57.8.Bm.good.wav
ffmpeg -ss 12737.9 -t 4.5 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:32:17.9.Bm.good.wav
ffmpeg -ss 12799.8 -t 4.6 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:33:19.8.Bm.good.wav
ffmpeg -ss 12854.3 -t 5.5 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:34:14.3.Bm.poor.wav
ffmpeg -ss 12923.1 -t 3.9 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:35:23.1.Bm.good.wav
ffmpeg -ss 12933.3 -t 4.6 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T03:35:33.3.Bm.good.wav
ffmpeg -

ffmpeg -ss 25351.9 -t 6.3 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:02:31.9.Bm.good.wav
ffmpeg -ss 25433.9 -t 14.0 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:03:53.9.Bm.good.wav
ffmpeg -ss 25572.0 -t 2.9 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:06:12.0.Bm.good.wav
ffmpeg -ss 25640.6 -t 4.2 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:07:20.6.Bm.good.wav
ffmpeg -ss 25840.2 -t 14.1 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:10:40.2.Bm.good.wav
ffmpeg -ss 25909.6 -t 5.9 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:11:49.6.Bm.good.wav
ffmpeg -ss 26012.1 -t 12.0 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:13:32.1.Bm.good.wav
ffmpeg -ss 26217.5 -t 15.4 -i wav/train/WAT_HZ_01_160217_000000.df100.x.wav read/Bm/WAT_HZ_2016-02-17T07:16:57.5.Bm.good.wav
ffmp

...

ffmpeg -ss 201020.4 -t 8.2 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T07:50:20.4.Bm.poor.wav
ffmpeg -ss 201103.3 -t 2.8 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T07:51:43.3.Bm.poor.wav
ffmpeg -ss 201325.9 -t 2.8 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T07:55:25.9.Bm.poor.wav
ffmpeg -ss 201339.9 -t 2.9 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T07:55:39.9.Bm.poor.wav
ffmpeg -ss 201711.0 -t 1.5 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T08:01:51.0.Bm.poor.wav
ffmpeg -ss 201918.7 -t 5.4 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T08:05:18.7.Bm.poor.wav
ffmpeg -ss 202240.6 -t 3.1 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T08:10:40.6.Bm.poor.wav
ffmpeg -ss 202282.6 -t 4.1 -i wav/train/WAT_BP_01_161002_000000.df100.x.wav read/Bm/WAT_BP_2016-10-04T08:11:22.6.Bm.poor.wav


ffmpeg -i wav/train/WAT_NC_01_150506_073845.df100.x.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '318075')
[] []
[]
wav/train/HAT_A_02_130224_000000.d100.x
HAT_A_02_130224_000000.d100.x
('date1 is ', '2013-02-24')
('start_date is ', datetime.datetime(2013, 2, 24, 0, 0))
ffmpeg -i wav/train/HAT_A_02_130224_000000.d100.x.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '272250')
[260347.2, 260642.6, 260666.9, 262456.2, 262492.3, 265632.6, 265651.7, 265712.8, 266050.7, 266132.1, 266284.2, 266767.0, 266962.2, 267095.4, 267205.7, 267963.2, 267980.4, 268313.1] [1.1, 1.3, 1.4, 1.5, 1.7, 1.8, 1.2, 1.5, 1.5, 1.2, 1.4, 1.2, 1.8, 1.3, 2.1, 1.6, 1.8, 1.4]
['2013-02-27T00:19:07.2', '2013-02-27T00:24:02.6', '2013-02-27T00:24:26.9', '2013-02-27T00:54:16.2', '2013-02-27T00:54:52.3', '2013-02-27T

ffmpeg -ss 19248.3 -t 2.3 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:20:48.3.Bm.poor.wav
ffmpeg -ss 20026.3 -t 4.0 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:33:46.3.Bm.poor.wav
ffmpeg -ss 20469.8 -t 5.1 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:41:09.8.Bm.good.wav
ffmpeg -ss 20506.2 -t 5.4 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:41:46.2.Bm.poor.wav
ffmpeg -ss 20538.8 -t 3.1 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:42:18.8.Bm.poor.wav
ffmpeg -ss 20897.5 -t 5.9 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:48:17.5.Bm.poor.wav
ffmpeg -ss 20967.8 -t 3.9 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T05:49:27.8.Bm.poor.wav
ffmpeg -ss 22559.9 -t 4.9 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-21T06:15:59.9.Bm.poor.wav
ffmpeg -ss 31896.2 -t 2.8 -i wav

ffmpeg -ss 98945.6 -t 1.9 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T03:29:05.6.Bm.poor.wav
ffmpeg -ss 99235.0 -t 6.3 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T03:33:55.0.Bm.poor.wav
ffmpeg -ss 99378.2 -t 6.2 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T03:36:18.2.Bm.poor.wav
ffmpeg -ss 99447.4 -t 8.0 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T03:37:27.4.Bm.poor.wav
ffmpeg -ss 99493.5 -t 3.1 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T03:38:13.5.Bm.poor.wav
ffmpeg -ss 100342.2 -t 3.1 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T03:52:22.2.Bm.poor.wav
ffmpeg -ss 101422.5 -t 4.8 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T04:10:22.5.Bm.poor.wav
ffmpeg -ss 102947.4 -t 4.1 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-22T04:35:47.4.Bm.poor.wav
ffmpeg -ss 103101.8 -t 2.4 -i

ffmpeg -ss 191067.2 -t 3.6 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T05:04:27.2.Bm.poor.wav
ffmpeg -ss 193899.0 -t 3.3 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T05:51:39.0.Bm.poor.wav
ffmpeg -ss 203653.8 -t 2.8 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T08:34:13.8.Bm.poor.wav
ffmpeg -ss 208634.8 -t 3.6 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T09:57:14.8.Bm.poor.wav
ffmpeg -ss 208926.1 -t 3.6 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T10:02:06.1.Bm.poor.wav
ffmpeg -ss 209109.0 -t 3.7 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T10:05:09.0.Bm.poor.wav
ffmpeg -ss 210108.5 -t 3.6 -i wav/train/HAT_A_02_121021_000000.d100.x.wav read/Bm/HAT_A_2012-10-23T10:21:48.5.Bm.poor.wav


In [4]:
import librosa
import util

def padding_audio(data,fs,T):
    # Calculate target number of samples
    N_tar = int(fs * T)
    # Calculate number of zero samples to append
    shape = data.shape
    # Create the target shape
    N_pad = N_tar - shape[0]
    print("Padding with %f seconds of silence" % float(N_pad/fs))
    shape = (N_pad,) + shape[1:]
    # Stack only if there is something to append
    if shape[0] > 0:
        if len(shape) > 1:
            return np.vstack((data,np.zeros(shape)))
        else:
            return np.hstack((data,np.zeros(shape)))
    else:
        return data




def read_audio(read_dir,sub_dirs,T_total,padding_dir,labels):
    for k, sub_dir in enumerate(sub_dirs):
        #print("label: %s" % (label))
        print("sub_dir: %s" % (sub_dir))
        for i, f in enumerate(glob.glob(read_dir + os.sep+ sub_dir + os.sep +'*.wav')):               # for each WAV file

            wavFile = f
            print wavFile
            waveFile_name = (os.path.splitext(wavFile)[0]).split(os.sep)[2]
            print waveFile_name
            quality = util.splitext(waveFile_name)[2]
            print quality
            # Read the wav file
            in_data, fs = librosa.load(wavFile)

            #print("fs is ",fs)
            #print("data is ", in_data)
            #print os.path.splitext(wavFile)[0]


            cmd = """ffmpeg -i """ + wavFile + """ 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'"""
            rs = sh.run(cmd, True)

            duration_in_wavFile = rs.stdout()
            print("the duration of wavfile is ", duration_in_wavFile)
            print type(duration_in_wavFile)

            if float(T_total) >= float(duration_in_wavFile):
                out_data = padding_audio(in_data,fs,T_total)

                # Save the output file
                for l in labels:
                    if quality == l:
                        out_wav = padding_dir +"/"+ l +"/"+ waveFile_name +".wav"
                    # if quality == "Eg":
                    #     out_wav = result_padding_dir +"/"+ eg_padding_dir +"/" +waveFile_name+".wav"

                    librosa.output.write_wav(out_wav, out_data, fs)


            else:
                int_num = int(float(duration_in_wavFile) / T_total)+1
                number = float(duration_in_wavFile) % T_total
                print number,int_num
                if number != 0:
                    T = T_total*int_num
                    out_data = padding_audio(in_data,fs,T)
                    for l in labels:
                        if quality == l:
                            out_wav = padding_dir +"/"+ l +"/"+ waveFile_name +".wav"
                        # if quality == "Eg":
                        #     out_wav = result_padding_dir +"/"+ eg_padding_dir +"/" +waveFile_name+".wav"

                        librosa.output.write_wav(out_wav, out_data, fs)




def cut_padding_audio(padding_dir,sub_dirs,T_total,redimension_dir,labels):
    for k, sub_padding in enumerate(sub_dirs):
        #print("label: %s" % (label))
        print("sub_padding: %s" % (sub_padding))
        for i, f in enumerate(glob.glob(padding_dir + os.sep + sub_padding + os.sep+'*.wav')):               # for each WAV file

            wavFile = f
            print wavFile
            waveFile_name = (os.path.splitext(wavFile)[0]).split(os.sep)[2]
            quality = util.splitext(waveFile_name)[2]
            print quality

            cmd = """ffmpeg -i """ + wavFile + """ 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'"""
            rs = sh.run(cmd, True)

            duration_in_wavFile = rs.stdout()
            print("the duration of wavfile is ", duration_in_wavFile)
            int_num = int(float(duration_in_wavFile) / float(T_total))-1
            print int_num
            cut_point = np.linspace(0,T_total*int_num,int_num+1)
            print cut_point
            if float(duration_in_wavFile) > float(T_total):
                for j in range(0, len(cut_point)):
                    print str(cut_point[j])
                    for l in labels:

                        if quality == l:
                            number = np.random.randint(0,9)
                            if number>2:
                                path_name = redimension_dir+"/"+"train" +"/"+ l +"/"
                            else:
                                path_name = redimension_dir+"/"+"test" +"/"+ l +"/"
                        # if quality == "Eg":
                        #     path_name = result_redimension_dir +"/"+ eg_redimension_dir +"/"

                    cmd = "ffmpeg -ss " + str(cut_point[j]) + " -t " + str(T_total)+ " -i " + wavFile + " " + path_name + waveFile_name +"_"+ str(j) + ".wav"
                    sh.run(cmd)
            else:
                for l in labels:
                    if quality == l:
                        number = np.random.randint(0,9)
                        if number>2:
                            path_name = redimension_dir+"/"+"train" +"/"+ l +"/"
                        else:
                            path_name = redimension_dir+"/"+"test" +"/"+ l +"/"
                    # if quality == "Eg":
                    #     path_name = result_redimension_dir +"/"+ eg_redimension_dir +"/"

                cmd = "cp " + wavFile +" "+ path_name
                sh.run(cmd)

def clean(file_dir):
    for i, f in enumerate(glob.glob(file_dir + os.sep +'*')):
        # print f
        cmd = "rm -rf " + f  + "/*.wav"
        sh.run(cmd)



if __name__ == "__main__":

    # wav_dir = "result_eg"
    read_dir = "read"
    padding_dir = "padding"
    redimension_dir = "redimension"
    redimension__train_dir = "redimension/train"
    redimension__test_dir = "redimension/test"
    labels = []
    T_total = 4
    sub_dirs = []
    for i, f in enumerate(glob.glob(read_dir + os.sep +'*')):
        f = os.path.splitext(f)[0]
        sub_dirs.append(f.split(os.sep)[1])
        labels.append(f.split(os.sep)[1])
    print labels

    clean(padding_dir)
    clean(redimension__train_dir)
    clean(redimension__test_dir)
    read_audio(read_dir,sub_dirs,T_total,padding_dir,labels)
    cut_padding_audio(padding_dir,sub_dirs,T_total,redimension_dir,labels)
    clean(padding_dir)


['Bm', 'Eg', 'Ba']
rm -rf padding/Bm/*.wav
rm -rf padding/Eg/*.wav
rm -rf padding/Ba/*.wav
rm -rf redimension/train/Bm/*.wav
rm -rf redimension/train/Eg/*.wav
rm -rf redimension/train/Ba/*.wav
rm -rf redimension/test/Bm/*.wav
rm -rf redimension/test/Eg/*.wav
rm -rf redimension/test/Ba/*.wav
sub_dir: Bm
read/Bm/WAT_BP_2016-10-03T03:38:37.3.Bm.poor.wav
WAT_BP_2016-10-03T03:38:37.3.Bm.poor
Bm
ffmpeg -i read/Bm/WAT_BP_2016-10-03T03:38:37.3.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '3.8')
<type 'str'>
Padding with 0.000000 seconds of silence
read/Bm/WAT_BP_2016-10-02T03:06:51.8.Bm.poor.wav
WAT_BP_2016-10-02T03:06:51.8.Bm.poor
Bm
ffmpeg -i read/Bm/WAT_BP_2016-10-02T03:06:51.8.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '2.7')
<type 'str'>
Padding with 1.000000 s

ffmpeg -i read/Bm/WAT_BP_2016-10-02T03:16:35.4.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '3.5')
<type 'str'>
Padding with 0.000000 seconds of silence
read/Bm/HAT_A_2012-10-25T17:08:35.0.Bm.poor.wav
HAT_A_2012-10-25T17:08:35.0.Bm.poor
Bm
ffmpeg -i read/Bm/HAT_A_2012-10-25T17:08:35.0.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '2.2')
<type 'str'>
Padding with 1.000000 seconds of silence
read/Bm/HAT_A_2012-10-24T13:33:36.0.Bm.poor.wav
HAT_A_2012-10-24T13:33:36.0.Bm.poor
Bm
ffmpeg -i read/Bm/HAT_A_2012-10-24T13:33:36.0.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4.8')
<type 'str'>
0.8 2
Padding with 3.000000 seconds of silence
read/Bm/HAT_A_2012

ffmpeg -i read/Bm/WAT_BP_2016-10-02T07:13:54.9.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '2.6')
<type 'str'>
Padding with 1.000000 seconds of silence
read/Bm/WAT_BP_2016-10-04T05:35:37.5.Bm.poor.wav
WAT_BP_2016-10-04T05:35:37.5.Bm.poor
Bm
ffmpeg -i read/Bm/WAT_BP_2016-10-04T05:35:37.5.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '8.1')
<type 'str'>
0.1 3
Padding with 3.000000 seconds of silence
read/Bm/WAT_HZ_2016-02-17T03:01:11.3.Bm.good.wav
WAT_HZ_2016-02-17T03:01:11.3.Bm.good
Bm
ffmpeg -i read/Bm/WAT_HZ_2016-02-17T03:01:11.3.Bm.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '3.1')
<type 'str'>
Padding with 0.000000 seconds of silence
read/Bm/WAT_

ffmpeg -i read/Bm/HAT_A_2012-10-24T23:46:04.1.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '6.5')
<type 'str'>
2.5 2
Padding with 1.000000 seconds of silence
read/Bm/HAT_A_2012-10-22T03:17:46.3.Bm.poor.wav
HAT_A_2012-10-22T03:17:46.3.Bm.poor
Bm
ffmpeg -i read/Bm/HAT_A_2012-10-22T03:17:46.3.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '3.5')
<type 'str'>
Padding with 0.000000 seconds of silence
read/Bm/WAT_BP_2016-10-02T06:00:04.4.Bm.poor.wav
WAT_BP_2016-10-02T06:00:04.4.Bm.poor
Bm
ffmpeg -i read/Bm/WAT_BP_2016-10-02T06:00:04.4.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '5.4')
<type 'str'>
1.4 2
Padding with 2.000000 seconds of silence
read/Bm/WA

ffmpeg -i read/Bm/WAT_BP_2016-10-03T04:11:24.3.Bm.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '6.1')
<type 'str'>
2.1 2
Padding with 1.000000 seconds of silence
read/Bm/WAT_HZ_2016-02-17T06:15:14.9.Bm.good.wav
WAT_HZ_2016-02-17T06:15:14.9.Bm.good
Bm
ffmpeg -i read/Bm/WAT_HZ_2016-02-17T06:15:14.9.Bm.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '11.3')
<type 'str'>
3.3 3
Padding with 0.000000 seconds of silence
read/Bm/WAT_HZ_2016-02-17T00:04:33.6.Bm.good.wav
WAT_HZ_2016-02-17T00:04:33.6.Bm.good
Bm
ffmpeg -i read/Bm/WAT_HZ_2016-02-17T00:04:33.6.Bm.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '7')
<type 'str'>
3.0 2
Padding with 1.000000 seconds of silence
r

...

ffmpeg -ss 0.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T02:33:53.3.Bm.poor.wav redimension/test/Bm/WAT_BP_2016-10-04T02:33:53.3.Bm.poor_0.wav
4.0
ffmpeg -ss 4.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T02:33:53.3.Bm.poor.wav redimension/train/Bm/WAT_BP_2016-10-04T02:33:53.3.Bm.poor_1.wav
padding/Bm/WAT_BP_2016-10-02T05:29:11.3.Bm.poor.wav
Bm
ffmpeg -i padding/Bm/WAT_BP_2016-10-02T05:29:11.3.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Bm/WAT_BP_2016-10-02T05:29:11.3.Bm.poor.wav redimension/test/Bm/
padding/Bm/WAT_BP_2016-10-02T07:04:03.9.Bm.poor.wav
Bm
ffmpeg -i padding/Bm/WAT_BP_2016-10-02T07:04:03.9.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Bm/WAT_BP_2016-10-02T07:04:03.9.Bm.poor.wav redimension/train/Bm/
padding/Bm/W

ffmpeg -i padding/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '12')
2
[0. 4. 8.]
0.0
ffmpeg -ss 0.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor.wav redimension/train/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor_0.wav
4.0
ffmpeg -ss 4.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor.wav redimension/train/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor_1.wav
8.0
ffmpeg -ss 8.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor.wav redimension/train/Bm/WAT_BP_2016-10-04T00:51:55.3.Bm.poor_2.wav
padding/Bm/HAT_A_2012-10-24T09:13:00.4.Bm.good.wav
Bm
ffmpeg -i padding/Bm/HAT_A_2012-10-24T09:13:00.4.Bm.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '8')
1
[0. 4.]
0.0
ffmpeg -ss 0.0 -t 4 -i padding/Bm/HAT_A_2012-10-24T09:13:00.

ffmpeg -ss 0.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T09:38:39.4.Bm.poor.wav redimension/train/Bm/WAT_BP_2016-10-04T09:38:39.4.Bm.poor_0.wav
4.0
ffmpeg -ss 4.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T09:38:39.4.Bm.poor.wav redimension/train/Bm/WAT_BP_2016-10-04T09:38:39.4.Bm.poor_1.wav
8.0
ffmpeg -ss 8.0 -t 4 -i padding/Bm/WAT_BP_2016-10-04T09:38:39.4.Bm.poor.wav redimension/test/Bm/WAT_BP_2016-10-04T09:38:39.4.Bm.poor_2.wav
padding/Bm/WAT_HZ_2016-02-17T06:26:30.7.Bm.good.wav
Bm
ffmpeg -i padding/Bm/WAT_HZ_2016-02-17T06:26:30.7.Bm.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '16')
3
[ 0.  4.  8. 12.]
0.0
ffmpeg -ss 0.0 -t 4 -i padding/Bm/WAT_HZ_2016-02-17T06:26:30.7.Bm.good.wav redimension/test/Bm/WAT_HZ_2016-02-17T06:26:30.7.Bm.good_0.wav
4.0
ffmpeg -ss 4.0 -t 4 -i padding/Bm/WAT_HZ_2016-02-17T06:26:30.7.Bm.good.wav redimension/train/Bm/WAT_HZ_2016-02-17T06:26:30.7.Bm.good_1.wav


ffmpeg -i padding/Eg/WAT_NC_2015-05-20T04:14:50.3.Eg.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Eg/WAT_NC_2015-05-20T04:14:50.3.Eg.poor.wav redimension/train/Eg/
padding/Eg/HAT_A_2013-02-27T01:55:32.1.Eg.good.wav
Eg
ffmpeg -i padding/Eg/HAT_A_2013-02-27T01:55:32.1.Eg.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Eg/HAT_A_2013-02-27T01:55:32.1.Eg.good.wav redimension/test/Eg/
padding/Eg/WAT_NC_2015-05-20T04:15:35.3.Eg.poor.wav
Eg
ffmpeg -i padding/Eg/WAT_NC_2015-05-20T04:15:35.3.Eg.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Eg/WAT_NC_2015-05-20T04:15:35.3.Eg.poor.wav redimension/train/E

ffmpeg -i padding/Eg/HAT_A_2013-02-27T02:06:07.0.Eg.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Eg/HAT_A_2013-02-27T02:06:07.0.Eg.poor.wav redimension/train/Eg/
padding/Eg/HAT_A_2013-02-27T00:54:16.2.Eg.poor.wav
Eg
ffmpeg -i padding/Eg/HAT_A_2013-02-27T00:54:16.2.Eg.poor.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Eg/HAT_A_2013-02-27T00:54:16.2.Eg.poor.wav redimension/train/Eg/
padding/Eg/HAT_A_2013-02-27T01:48:32.8.Eg.good.wav
Eg
ffmpeg -i padding/Eg/HAT_A_2013-02-27T01:48:32.8.Eg.good.wav 2>&1 | grep "Duration"| cut -d ' ' -f 4 | sed s/,// | awk '{ split($1, A, ":"); print 3600*A[1] + 60*A[2] + A[3] }'
('the duration of wavfile is ', '4')
0
[0.]
cp padding/Eg/HAT_A_2013-02-27T01:48:32.8.Eg.good.wav redimension/train/Eg/
s

In [7]:
import math
from sklearn import preprocessing
import pywt


def  get_teager_energy(wp,Node,n,N_limit):
    k_len = 12
    TC = []
    for k in range(k_len):
        sum = 0
        for l in range(2**n):
            el = get_e(l,N_limit,wp,Node)
            sum += math.log(el,2) * math.cos(k*(l-0.5)*math.pi/(2**n))
        TC.append(sum)
    return TC

def get_e(l,N_limit,wp,Node):
    name = Node[l]
    x = wp[name].data
    sum = 0
    for t in range(N_limit):

        sum += x[t+1]*x[t+1]-x[t]*x[t+2]
        #print sum
    el = float(sum/N_limit)
    return el

def extration_wavelet_packet(wavFile):

    sig, fs = librosa.load(wavFile)
    print("fs is ",fs)
    print("signal length is ", len(sig))
    N_signal = len(sig)
    n_level = 6

    wp = pywt.WaveletPacket(data=sig, wavelet='db3', mode='symmetric')

    Node = []
    N_limit = 16
    limit = 0
    for node in wp.get_level(6, 'natural'):
            Node.append(node.path)

    print("sub sample is ",len(Node))

    TC = get_teager_energy(wp,Node,n_level,N_limit)
    print ("TC is ",TC)
    return TC


def extration_mfcc(wavFile):

    # Read the wav file
    X, sample_rate = librosa.load(wavFile)
    print("sample_rate  is ",sample_rate )
    print("data is ", X)

    # sftf
    stft = np.abs(librosa.stft(X))

    # mfcc
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T,axis=0)
    '''
    # chroma
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # melspectrogram
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    return mfccs,chroma,mel,contrast
    '''
    return mfccs

def extration_rawSignal(wavFile,max_len,sample_rate):

    # Read the wav file
    X, sample_rate = librosa.load(wavFile, sr=sample_rate, mono=True)
    # print("sample_rate  is ",sample_rate )
    # print("data is ", X)
    # print X.shape
    if X.shape[0] < max_len:
        pad = np.zeros(max_len - X.shape[0])
        # print("pad is ",pad)
        X = np.hstack((X, pad))
    elif X.shape[0] > max_len:
        X = X[:max_len]
    # print X.shape
    return X



def parse_audio_files_rawSignal(redimension_dir,sub_read,file_ext):
    max_len=200
    sample_rate=100
    features, labels = np.empty((0,max_len)), np.empty(0)
    for label, sub_dir in enumerate(sub_read):
        print("label: %s" % (label))
        #print("sub_dir: %s" % (sub_dir))
        for f in glob.glob(os.path.join(redimension_dir, sub_dir, file_ext)):
            print("extract file: %s" % (f))
            waveFile_name = (os.path.splitext(f)[0]).split(os.sep)[2]
            try:
                audio = extration_rawSignal(f,max_len,sample_rate)

            except Exception as e:
                print("[Error] extract feature error. %s" % (e))
                continue

            ext_features = np.hstack([audio])
            #print len(ext_features)
            #print ext_features
            features = np.vstack([features,ext_features])
            # labels = np.append(labels, quality)
            labels = np.append(labels, label)
        # print (features.shape)
        #print features
        #print labels
    return np.array(features), np.array(labels, dtype = np.int)


def parse_audio_files_waveletPackets(read_dir,sub_read,file_ext):
    features, labels = np.empty((0,12)), np.empty(0)
    for label, sub_dir in enumerate(sub_read):
        print("label: %s" % (label))
        #print("sub_dir: %s" % (sub_dir))
        for f in glob.glob(os.path.join(read_dir, sub_dir, file_ext)):
            print("extract file: %s" % (f))
            waveFile_name = (os.path.splitext(f)[0]).split(os.sep)[2]
            quality = util.splitext(waveFile_name)[2]
            try:
                TC = extration_wavelet_packet(f)

            except Exception as e:
                print("[Error] extract feature error. %s" % (e))
                continue

            ext_features = np.hstack([TC])
            #print len(ext_features)
            #print ext_features
            features = np.vstack([features,ext_features])
            # labels = np.append(labels, quality)
            labels = np.append(labels, label)
        print (features.shape)
        #print features
        #print labels
    return np.array(features), np.array(labels, dtype = np.int)


def parse_audio_files_mfcc(read_dir,sub_read,file_ext):
    features, labels = np.empty((0,13)), np.empty(0)
    for label, sub_dir in enumerate(sub_read):
        print("label: %s" % (label))
        #print("sub_dir: %s" % (sub_dir))
        for f in glob.glob(os.path.join(read_dir, sub_dir, file_ext)):
            print("extract file: %s" % (f))
            waveFile_name = (os.path.splitext(f)[0]).split(os.sep)[2]
            quality = util.splitext(waveFile_name)[2]
            try:
                mfccs = extration_mfcc(f)
                # print len(mfccs),len(chroma),len(mel),len(contrast)
                print (len(mfccs))
                #print ("mfcc is",np.array(mfccs))
            except Exception as e:
                print("[Error] extract feature error. %s" % (e))
                continue

            # ext_features = np.hstack([mfccs,chroma,mel,contrast])
            ext_features = np.hstack([mfccs])
            #print len(ext_features)
            #print ext_features
            features = np.vstack([features,ext_features])
            # labels = np.append(labels, quality)
            labels = np.append(labels, label)
        print (features.shape)
        print (features)
        #print labels
    return np.array(features), np.array(labels, dtype = np.int)


def normaliser_features(features):

    features_normalisation = preprocessing.scale(features)

    return features_normalisation



def encode_label(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    labels_encode = np.zeros((n_labels,1))
    #print labels_encode
    for i in range(n_labels):
        labels_encode[i]= labels[i]

    return labels_encode


if __name__ == "__main__":

    read_dir = "read"
    sub_read = ['Bm','Eg']
    file_ext='*.wav'

    cmd = "rm -rf feature/*"
    sh.run(cmd)
    ############ mfcc
    
    features_mfcc,labels_mfcc = parse_audio_files_mfcc(read_dir,sub_read,file_ext)
    features_mfcc = normaliser_features(features_mfcc)
    # print ("features noramallisation are ",features_normalisation)
    labels_mfcc = encode_label(labels_mfcc)
    # print ("label encode is ",labels_encode)
    #print type(labels_encode)

    np.savetxt("feature/train_features_mfcc.txt",features_mfcc)
    np.savetxt("feature/train_label_mfcc.txt",labels_mfcc)

    ############ wavelet
    features_wavelet,labels_wavelet = parse_audio_files_waveletPackets(read_dir,sub_read,file_ext)
    features_wavelet = normaliser_features(features_wavelet)
    labels_wavelet = encode_label(labels_wavelet)

    np.savetxt("feature/train_features_wavelet.txt",features_wavelet)
    np.savetxt("feature/train_label_wavelet.txt",labels_wavelet)
    
    ###########  raw signal
    features_rawSignal,labels_rawSignal = parse_audio_files_rawSignal(read_dir,sub_read,file_ext)
    features_rawSignal = normaliser_features(features_rawSignal)
    labels_rawSignal = encode_label(labels_rawSignal)
    
    np.savetxt("feature/train_features_rawSignal.txt",features_rawSignal)
    np.savetxt("feature/train_label_rawSignal.txt",labels_rawSignal)

rm -rf feature/*
label: 0
extract file: read/Bm/WAT_BP_2016-10-03T03:38:37.3.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([2.8123564e-04, 2.8524868e-04, 2.8487126e-04, ..., 2.1697747e-04,
       1.3340671e-04, 6.3469983e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-02T03:06:51.8.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-5.8040587e-04, -6.7321921e-04, -7.6680986e-04, ...,
       -7.5789139e-05, -5.5830558e-05, -3.8404924e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-02T02:22:28.4.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-0.00021329, -0.0002352 , -0.00025005, ...,  0.00048506,
        0.00033012,  0.00018556], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-04T00:35:04.8.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([5.7343382e-04, 5.8472640e-04, 5.8663310e-04, ..., 8.6585744e-05,
       5.9645630e-05, 3.7410002e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-02T04:26:34

('sample_rate  is ', 22050)
('data is ', array([-7.4460260e-03, -7.8225685e-03, -8.1001185e-03, ...,
       -8.7231041e-05, -7.0822556e-05, -5.0797324e-05], dtype=float32))
13
extract file: read/Bm/WAT_HZ_2016-02-17T00:16:37.5.Bm.good.wav
('sample_rate  is ', 22050)
('data is ', array([-2.6872967e-04, -3.1955738e-04, -3.7016050e-04, ...,
       -2.4588659e-04, -1.3040770e-04, -3.1280280e-05], dtype=float32))
13
extract file: read/Bm/HAT_A_2012-10-22T03:26:06.0.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([0.00046827, 0.00046432, 0.00045408, ..., 0.00039673, 0.00027717,
       0.00016483], dtype=float32))
13
extract file: read/Bm/HAT_A_2012-10-25T07:56:40.8.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([ 0.00024973,  0.00027439,  0.00029339, ..., -0.00045721,
       -0.00037234, -0.00027375], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-02T02:57:00.9.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([5.4603617e-04, 5.0255173e-04, 4.5400590e

('sample_rate  is ', 22050)
('data is ', array([ 5.1551444e-05,  6.5808868e-05,  7.8563928e-05, ...,
       -5.1129977e-05, -4.1636016e-05, -3.3306733e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-02T06:52:20.0.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([ 7.1902131e-04,  7.7408279e-04,  8.2143303e-04, ...,
        4.8205744e-05,  1.4186774e-05, -1.0822235e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-04T02:14:57.9.Bm.good.wav
('sample_rate  is ', 22050)
('data is ', array([ 0.00078214,  0.00081033,  0.00081972, ..., -0.0006434 ,
       -0.00044892, -0.00026651], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-04T04:11:29.5.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-4.5757148e-05, -5.5204717e-05, -5.8888992e-05, ...,
        2.1396087e-04,  1.4565446e-04,  8.3707913e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-04T06:20:56.3.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-0.00067911, -

('sample_rate  is ', 22050)
('data is ', array([3.4765940e-04, 3.5913687e-04, 3.6865781e-04, ..., 5.8654638e-05,
       5.1673498e-05, 4.4342123e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-03T06:12:56.4.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-1.5533509e-05, -4.8840873e-05, -8.6030261e-05, ...,
       -3.0666928e-05, -2.5022915e-05, -1.9333949e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-03T00:55:14.1.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([1.0062004e-03, 1.0595433e-03, 1.0997518e-03, ..., 1.0917872e-04,
       7.6626588e-05, 4.6279172e-05], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-04T06:27:51.9.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-0.00196029, -0.00209927, -0.00220815, ..., -0.00037215,
       -0.00026233, -0.00016228], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-03T03:22:45.7.Bm.good.wav
('sample_rate  is ', 22050)
('data is ', array([-1.1502192e-03, -1.2091061

('sample_rate  is ', 22050)
('data is ', array([-4.3010720e-04, -4.5839537e-04, -4.8146670e-04, ...,
       -3.4216737e-05, -1.5306528e-05, -5.6221421e-07], dtype=float32))
13
extract file: read/Bm/HAT_A_2012-10-25T16:33:54.4.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-0.0002393 , -0.00028066, -0.00032105, ..., -0.00028871,
       -0.00019599, -0.00011008], dtype=float32))
13
extract file: read/Bm/WAT_BP_2016-10-03T22:03:44.9.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-1.2866390e-03, -1.2690080e-03, -1.2210418e-03, ...,
       -1.0369050e-04, -9.8701850e-05, -8.8036657e-05], dtype=float32))
13
extract file: read/Bm/HAT_A_2012-10-22T12:39:34.3.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-0.00069599, -0.00072469, -0.00073889, ..., -0.00040591,
       -0.00025736, -0.00012619], dtype=float32))
13
extract file: read/Bm/HAT_A_2012-10-24T19:40:25.9.Bm.poor.wav
('sample_rate  is ', 22050)
('data is ', array([-6.4991653e-04, -6.8234425e-04, -7.

...

extract file: read/Bm/HAT_A_2012-10-27T09:07:01.0.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-27T06:32:43.5.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-24T04:08:15.3.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-04T01:50:55.8.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-03T06:48:51.7.Bm.good.wav
extract file: read/Bm/WAT_HZ_2016-02-17T00:22:29.9.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T07:43:22.4.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-04T02:21:27.0.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-04T08:26:25.4.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T04:39:16.1.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-25T00:24:06.5.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T04:42:42.5.Bm.poor.wav
extract file: read/Bm/WAT_HZ_2016-02-17T03:22:25.5.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-21T05:49:27.8.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-23T01:02:55.5.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-03T04:01:27.1.Bm.good.wa

extract file: read/Bm/WAT_BP_2016-10-02T04:23:43.0.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-04T01:30:50.9.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-24T14:08:50.2.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T07:23:54.1.Bm.good.wav
extract file: read/Bm/WAT_HZ_2016-02-17T03:16:24.9.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T05:42:31.2.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T09:09:36.6.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T12:10:07.1.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-23T10:21:48.5.Bm.poor.wav
extract file: read/Bm/WAT_HZ_2016-02-17T01:42:14.9.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T09:17:44.4.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T07:24:55.8.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T12:48:25.0.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-04T06:04:08.7.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-24T18:38:03.9.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T01:30:05.6.Bm.poor

extract file: read/Bm/WAT_BP_2016-10-04T05:28:26.6.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T06:21:13.0.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-24T18:42:36.3.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T04:10:16.6.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T07:24:22.0.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-04T05:41:42.7.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-04T00:04:25.0.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T06:21:31.8.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T02:28:30.6.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T04:42:16.1.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T08:07:01.3.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T02:29:35.8.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-21T21:50:06.0.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T21:30:32.9.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-04T00:16:10.2.Bm.poor.wav
extract file: read/Bm/WAT_HZ_2016-02-17T21:14:35.0.Bm.goo

extract file: read/Bm/WAT_BP_2016-10-03T01:43:57.9.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T03:29:53.2.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-24T13:01:45.1.Bm.good.wav
extract file: read/Bm/WAT_HZ_2016-02-17T01:43:18.7.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T02:37:54.9.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-22T11:38:40.3.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T03:46:22.6.Bm.poor.wav
extract file: read/Bm/WAT_HZ_2016-02-17T18:10:30.2.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-24T05:21:36.6.Bm.poor.wav
extract file: read/Bm/WAT_HZ_2016-02-17T03:11:21.5.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-03T03:39:38.4.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-03T07:12:15.3.Bm.good.wav
extract file: read/Bm/WAT_BP_2016-10-02T02:30:16.7.Bm.poor.wav
extract file: read/Bm/HAT_A_2012-10-24T12:15:16.8.Bm.good.wav
extract file: read/Bm/HAT_A_2012-10-24T19:10:07.0.Bm.poor.wav
extract file: read/Bm/WAT_BP_2016-10-02T10:51:18.6.Bm.poor.w

extract file: read/Eg/HAT_A_2013-02-27T01:58:04.2.Eg.good.wav
extract file: read/Eg/HAT_A_2013-02-27T02:06:07.0.Eg.poor.wav
extract file: read/Eg/HAT_A_2013-02-27T00:54:16.2.Eg.poor.wav
extract file: read/Eg/HAT_A_2013-02-27T01:48:32.8.Eg.good.wav
